<a href="https://colab.research.google.com/github/sunshuofeng/SUN-s-REID/blob/master/REID_TEST1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#加载kaggle文件

In [ ]:
!pip install kaggle

In [ ]:
import json
token = {"username":"ssfailearning","key":"bccedfffce653f07d5acaf4409341c47"}
with open('/content/kaggle.json', 'w') as file:
  json.dump(token, file)

In [ ]:
!mkdir -p ~/.kaggle


In [ ]:
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v /content

- path is now set to: /content


In [ ]:
!kaggle datasets download -d rayiooo/reid_market-1501

reid_market-1501.zip: Skipping, found more recently modified local copy (use --force to force download)


# <font color=red>***配置文件***</font>

In [ ]:
cfg={
    'train_size':[384,128],
    'test_size':[384,128],
    'padding':10,
    'mean':[0.485, 0.456, 0.406],
    'std':[0.229, 0.224, 0.225],

    'Multi_Data':False,
    'num_workers':8,
    'SAMPLER':'',

    'train_bs':48,
    'test_bs':48,

    'train_K_instances':12,
     
    'epochs':70,
    'ckpt':'reid_test1.pt',
    'logpt':'test1.log',

    'rerank':False,
   
    'lr':3e-4,
    'momentum':0.9,
    'weight_decay':5e-4,
     
    'margin':0.3,
     
    'steps':[35,55],
    'warmup_iters':10,
    'warmup_factors':1/3,
    'gamma':0.1,
     
     

    'amp':True,
    'ema':False,
    'swa':False
    
}

# <font color=red>***读取数据***</font>

##<font color=blue>***取market1501***</font>

In [ ]:
import os
import zipfile
# os.makedirs('datasets')
zip_file = zipfile.ZipFile('/content/datasets/rayiooo/reid_market-1501/reid_market-1501.zip')
for names in zip_file.namelist():
        zip_file.extract(names,'datasets')
zip_file.close()

In [ ]:
import numpy as np
import pandas as pd
from torch.cuda.amp import GradScaler,autocast
import torch


##<font color=blue>***数据增强***</font>

###<font color=green>***图像***</font>

#### <font color=yellow>***随机擦除***</font>

In [ ]:
import math
import random


class RandomErasing(object):
    """ Randomly selects a rectangle region in an image and erases its pixels.
        'Random Erasing Data Augmentation' by Zhong et al.
        See https://arxiv.org/pdf/1708.04896.pdf
    Args:
         probability: The probability that the Random Erasing operation will be performed.
         sl: Minimum proportion of erased area against input image.
         sh: Maximum proportion of erased area against input image.
         r1: Minimum aspect ratio of erased area.
         mean: Erasing value.
    """

    def __init__(self, probability=0.5, sl=0.02, sh=0.4, r1=0.3, mean=(0.4914, 0.4822, 0.4465)):
        self.probability = probability
        self.mean = mean
        self.sl = sl
        self.sh = sh
        self.r1 = r1

    def __call__(self, img):

        if random.uniform(0, 1) >= self.probability:
            return img

        for attempt in range(100):
            area = img.size()[1] * img.size()[2]

            target_area = random.uniform(self.sl, self.sh) * area
            aspect_ratio = random.uniform(self.r1, 1 / self.r1)

            h = int(round(math.sqrt(target_area * aspect_ratio)))
            w = int(round(math.sqrt(target_area / aspect_ratio)))

            if w < img.size()[2] and h < img.size()[1]:
                x1 = random.randint(0, img.size()[1] - h)
                y1 = random.randint(0, img.size()[2] - w)
                if img.size()[0] == 3:
                    img[0, x1:x1 + h, y1:y1 + w] = self.mean[0]
                    img[1, x1:x1 + h, y1:y1 + w] = self.mean[1]
                    img[2, x1:x1 + h, y1:y1 + w] = self.mean[2]
                else:
                    img[0, x1:x1 + h, y1:y1 + w] = self.mean[0]
                return img

        return img

####<font color=yellow>***其余数据增强（torchvision transformss）***</font>



In [ ]:

import torchvision.transforms as T



def build_transforms(cfg, training=True):
    normalize_transform = T.Normalize(mean=cfg['mean'], std=cfg['std'])
    if training:
        transform = T.Compose([
            T.Resize(cfg['train_size']),
            T.RandomHorizontalFlip(p=0.5),
            T.Pad(cfg['padding']),
            T.RandomCrop(cfg['train_size']),
            T.ToTensor(),
            normalize_transform,
            RandomErasing(probability=0.5, mean=cfg['mean'])
        ])
    else:
        transform = T.Compose([
            T.Resize(cfg['test_size']),
            T.ToTensor(),
            normalize_transform
        ])

    return transform



##<font color=blue>***sampler***</font>

<font color=red>抽取N个pid(即N个人)，每个人抽取K张图片，目的是为了能确保至少有三元组，**锚，正样本，负样本**</font>


###<font color=green>***图像***</font>

In [ ]:

import copy
import random
import torch
from collections import defaultdict

import numpy as np
from torch.utils.data.sampler import Sampler

class RandomIdentitySampler(Sampler):
    """
    Randomly sample N identities, then for each identity,
    randomly sample K instances, therefore batch size is N*K.
    Args:
    - data_source (list): list of (img_path, pid, camid).
    - num_instances (int): number of instances per identity in a batch.
    - batch_size (int): number of examples in a batch.
    """

    def __init__(self, data_source, batch_size, num_instances):
        self.data_source = data_source
        self.batch_size = batch_size
        self.num_instances = num_instances
        self.num_pids_per_batch = self.batch_size // self.num_instances
        self.index_dic = defaultdict(list)
        for index, (_, pid, _) in enumerate(self.data_source):
            self.index_dic[pid].append(index)
        self.pids = list(self.index_dic.keys())

        # estimate number of examples in an epoch
        self.length = 0
        for pid in self.pids:
            idxs = self.index_dic[pid]
            num = len(idxs)
            if num < self.num_instances:
                num = self.num_instances
            self.length += num - num % self.num_instances

    def __iter__(self):
        batch_idxs_dict = defaultdict(list)

        for pid in self.pids:
            idxs = copy.deepcopy(self.index_dic[pid])
            if len(idxs) < self.num_instances:
                idxs = np.random.choice(idxs, size=self.num_instances, replace=True)
            random.shuffle(idxs)
            batch_idxs = []
            for idx in idxs:
                batch_idxs.append(idx)
                if len(batch_idxs) == self.num_instances:
                    batch_idxs_dict[pid].append(batch_idxs)
                    batch_idxs = []

        avai_pids = copy.deepcopy(self.pids)
        final_idxs = []

        while len(avai_pids) >= self.num_pids_per_batch:
            selected_pids = random.sample(avai_pids, self.num_pids_per_batch)
            for pid in selected_pids:
                batch_idxs = batch_idxs_dict[pid].pop(0)
                final_idxs.extend(batch_idxs)
                if len(batch_idxs_dict[pid]) == 0:
                    avai_pids.remove(pid)

        self.length = len(final_idxs)
        return iter(final_idxs)

    def __len__(self):
        return self.length


##<font color=blue>***建立数据集***</font>

###<font color=green>***图像***</font>

####<font color=yellow>***图像读取数据集（market1501）***</font>

In [ ]:
import glob
import re
import os.path as osp
class Market1501():
    def __init__(self, root='/content/datasets', verbose=True, **kwargs):
        super(Market1501, self).__init__()
        self.dataset_dir = root
        self.train_dir = osp.join(self.dataset_dir, 'bounding_box_train')
        self.query_dir = osp.join(self.dataset_dir, 'query')
        self.gallery_dir = osp.join(self.dataset_dir, 'bounding_box_test')
        self._check_before_run()

        train = self._process_dir(self.train_dir, relabel=True)
        query = self._process_dir(self.query_dir, relabel=False)
        gallery = self._process_dir(self.gallery_dir, relabel=False)

        if verbose:
            print("=> Market1501 loaded")
            self.print_dataset_statistics(train, query, gallery)

        self.train = train
        self.query = query
        self.gallery = gallery

        self.num_train_pids, self.num_train_imgs, self.num_train_cams = self.get_imagedata_info(self.train)
        self.num_query_pids, self.num_query_imgs, self.num_query_cams = self.get_imagedata_info(self.query)
        self.num_gallery_pids, self.num_gallery_imgs, self.num_gallery_cams = self.get_imagedata_info(self.gallery)

    def print_dataset_statistics(self, train, query, gallery):
        num_train_pids, num_train_imgs, num_train_cams = self.get_imagedata_info(train)
        num_query_pids, num_query_imgs, num_query_cams = self.get_imagedata_info(query)
        num_gallery_pids, num_gallery_imgs, num_gallery_cams = self.get_imagedata_info(gallery)

        print("Dataset statistics:")
        print("  ----------------------------------------")
        print("  subset   | # ids | # images | # cameras")
        print("  ----------------------------------------")
        print("  train    | {:5d} | {:8d} | {:9d}".format(num_train_pids, num_train_imgs, num_train_cams))
        print("  query    | {:5d} | {:8d} | {:9d}".format(num_query_pids, num_query_imgs, num_query_cams))
        print("  gallery  | {:5d} | {:8d} | {:9d}".format(num_gallery_pids, num_gallery_imgs, num_gallery_cams))
        print("  ----------------------------------------")
    
    def get_imagedata_info(self, data):
        pids, cams = [], []
        for _, pid, camid in data:
            pids += [pid]
            cams += [camid]
        pids = set(pids)
        cams = set(cams)
        num_pids = len(pids)
        num_cams = len(cams)
        num_imgs = len(data)
        return num_pids, num_imgs, num_cams
    

    def _check_before_run(self):
        """Check if all files are available before going deeper"""
        if not osp.exists(self.dataset_dir):
            raise RuntimeError("'{}' is not available".format(self.dataset_dir))
        if not osp.exists(self.train_dir):
            raise RuntimeError("'{}' is not available".format(self.train_dir))
        if not osp.exists(self.query_dir):
            raise RuntimeError("'{}' is not available".format(self.query_dir))
        if not osp.exists(self.gallery_dir):
            raise RuntimeError("'{}' is not available".format(self.gallery_dir))

    
    ##最终返回的是每张图片的路径，pid，camid
    def _process_dir(self, dir_path, relabel=False):
        img_paths = glob.glob(osp.join(dir_path, '*.jpg'))

        ##匹配文件名中的有用信息，0001_c1代表pid=1，cid=1，
        ##至于为什么是[-\d]+,是因为有-1_c1这种的存在，也应该匹配上
        pattern = re.compile(r'([-\d]+)_c(\d)')

        pid_container = set()
        for img_path in img_paths:
            pid, _ = map(int, pattern.search(img_path).groups())
            if pid == -1: continue  # junk images are just ignored
            pid_container.add(pid)
        pid2label = {pid: label for label, pid in enumerate(pid_container)}

        dataset = []
        for img_path in img_paths:
            pid, camid = map(int, pattern.search(img_path).groups())
            if pid == -1: continue  # junk images are just ignored
            assert 0 <= pid <= 1501  # pid == 0 means background
            assert 1 <= camid <= 6
            camid -= 1  # index starts from 0
            if relabel: pid = pid2label[pid]
            dataset.append((img_path, pid, camid))

        return dataset


import torch.utils.data as Data
from PIL import Image


class ImageDataset(Data.Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img_path, pid, camid = self.dataset[index]
        img = Image.open(img_path)
        if self.transform is not None:
            img = self.transform(img)
        return img, pid, camid, img_path



###<font color=green>***建立dataloader***</font>

In [ ]:

###对于训练的时候我们不需要关心摄像头的问题，我们只需要让模型能认出图片里的人就行
def train_collate_fn(batch):
    imgs, pids, _, _, = zip(*batch)
    pids = torch.tensor(pids, dtype=torch.int64)
    return torch.stack(imgs, dim=0), pids

###对于验证集而言，为了提高验证的真实性，我们应该防止同一摄像头的图片进入验证（同一摄像头相当于数据泄露）
def val_collate_fn(batch):
    imgs, pids, camids, _ = zip(*batch)
    return torch.stack(imgs, dim=0), pids, camids



def make_data_loader(datasets,cfg):
    train_transform=build_transforms(cfg,training=True)
    test_transform=build_transforms(cfg,training=False)
    if cfg['Multi_Data']:
        data=datasets[0]
        for dataset in datasets[1:]:
            data.train.extend(dataset.train)
            data.query.extend(dataset.query)
            data.gallery.extend(dataset.gallery)

            data.num_train_pids+=dataset.num_train_pids
    else:
        data=datasets

    num_classes=data.num_train_pids
    train_set=ImageDataset(data.train,train_transform)

    if cfg['SAMPLER']=='softmax':
        train_loader=Data.DataLoader(train_set,batch_size=cfg['train_bs'],
                                     shuffle=True,collate_fn=train_collate_fn)
    else:
        train_loader=Data.DataLoader(train_set,batch_size=cfg['train_bs'],
                                     sampler=RandomIdentitySampler(data.train,
                                    cfg['train_bs'],cfg['train_K_instances']),
                                    num_workers=cfg['num_workers'],
                                    collate_fn=train_collate_fn)
    
    val_set = ImageDataset(data.query + data.gallery, test_transform)
    val_loader = Data.DataLoader(
        val_set, batch_size=cfg['test_bs'], shuffle=False, num_workers=cfg['num_workers'],
        collate_fn=val_collate_fn
    )
    return train_loader, val_loader, len(data.query), num_classes



#<font color=red>***模型建立***</font>

##<font color=blue>***图像***</font>

###<font color=green>***strong-baseline***</font>

In [ ]:
from torchvision.models import resnet50
class Baseline(nn.Module):
    in_planes = 2048
    def __init__(self, num_classes, neck='bnneck', neck_feat='after'):
        super(Baseline, self).__init__()
#         if model_name == 'resnet18':
#             self.in_planes = 512
#             self.base = ResNet(last_stride=last_stride, 
#                                block=BasicBlock, 
#                                layers=[2, 2, 2, 2])
#         elif model_name == 'resnet34':
#             self.in_planes = 512
#             self.base = ResNet(last_stride=last_stride,
#                                block=BasicBlock,
#                                layers=[3, 4, 6, 3])
#         elif model_name == 'resnet50':
#             self.base = ResNet(last_stride=last_stride,
#                                block=Bottleneck,
#                                layers=[3, 4, 6, 3])
#         elif model_name == 'resnet101':
#             self.base = ResNet(last_stride=last_stride,
#                                block=Bottleneck, 
#                                layers=[3, 4, 23, 3])
#         elif model_name == 'resnet152':
#             self.base = ResNet(last_stride=last_stride, 
#                                block=Bottleneck,
#                                layers=[3, 8, 36, 3])
            
#         elif model_name == 'se_resnet50':
#             self.base = SENet(block=SEResNetBottleneck, 
#                               layers=[3, 4, 6, 3], 
#                               groups=1, 
#                               reduction=16,
#                               dropout_p=None, 
#                               inplanes=64, 
#                               input_3x3=False,
#                               downsample_kernel_size=1, 
#                               downsample_padding=0,
#                               last_stride=last_stride) 
#         elif model_name == 'se_resnet101':
#             self.base = SENet(block=SEResNetBottleneck, 
#                               layers=[3, 4, 23, 3], 
#                               groups=1, 
#                               reduction=16,
#                               dropout_p=None, 
#                               inplanes=64, 
#                               input_3x3=False,
#                               downsample_kernel_size=1, 
#                               downsample_padding=0,
#                               last_stride=last_stride)
#         elif model_name == 'se_resnet152':
#             self.base = SENet(block=SEResNetBottleneck, 
#                               layers=[3, 8, 36, 3],
#                               groups=1, 
#                               reduction=16,
#                               dropout_p=None, 
#                               inplanes=64, 
#                               input_3x3=False,
#                               downsample_kernel_size=1, 
#                               downsample_padding=0,
#                               last_stride=last_stride)  
#         elif model_name == 'se_resnext50':
#             self.base = SENet(block=SEResNeXtBottleneck,
#                               layers=[3, 4, 6, 3], 
#                               groups=32, 
#                               reduction=16,
#                               dropout_p=None, 
#                               inplanes=64, 
#                               input_3x3=False,
#                               downsample_kernel_size=1, 
#                               downsample_padding=0,
#                               last_stride=last_stride) 
#         elif model_name == 'se_resnext101':
#             self.base = SENet(block=SEResNeXtBottleneck,
#                               layers=[3, 4, 23, 3], 
#                               groups=32, 
#                               reduction=16,
#                               dropout_p=None, 
#                               inplanes=64, 
#                               input_3x3=False,
#                               downsample_kernel_size=1, 
#                               downsample_padding=0,
#                               last_stride=last_stride)
#         elif model_name == 'senet154':
#             self.base = SENet(block=SEBottleneck, 
#                               layers=[3, 8, 36, 3],
#                               groups=64, 
#                               reduction=16,
#                               dropout_p=0.2, 
#                               last_stride=last_stride)
#         elif model_name == 'resnet50_ibn_a':
#             self.base = resnet50_ibn_a(last_stride)

#         if pretrain_choice == 'imagenet':
#             self.base.load_param(model_path)
#             print('Loading pretrained ImageNet model......')
        self.base=resnet50(True)
        self.base=nn.Sequential(
            self.base.conv1,
            self.base.bn1,
            self.base.maxpool,
            self.base.layer1,
            self.base.layer2,
            self.base.layer3,
            self.base.layer4
        )
        self.gap = nn.AdaptiveAvgPool2d(1)
        # self.gap = nn.AdaptiveMaxPool2d(1)
        self.num_classes = num_classes
        self.neck = neck
        self.neck_feat = neck_feat

        if self.neck == 'no':
            self.classifier = nn.Linear(self.in_planes, self.num_classes)
            # self.classifier = nn.Linear(self.in_planes, self.num_classes, bias=False)     # new add by luo
            # self.classifier.apply(weights_init_classifier)  # new add by luo
        elif self.neck == 'bnneck':
            self.bottleneck = nn.BatchNorm1d(self.in_planes)
            self.bottleneck.bias.requires_grad_(False)  # no shift
            self.classifier = nn.Linear(self.in_planes, self.num_classes, bias=False)

            self.bottleneck.apply(weights_init_kaiming)
            self.classifier.apply(weights_init_classifier)

    def forward(self, x):

        global_feat = self.gap(self.base(x))  # (b, 2048, 1, 1)
        global_feat = global_feat.view(global_feat.shape[0], -1)  # flatten to (bs, 2048)
        out={}
        if self.neck == 'no':
            feat = global_feat
        elif self.neck == 'bnneck':
            feat = self.bottleneck(global_feat)  # normalize for angular softmax

        if self.training:
            cls_score = self.classifier(feat)
            out['logit']=cls_score
            out['feat']=global_feat
            return out  # global feature for triplet loss
        else:
            if self.neck_feat == 'after':
                # print("Test with feature after BN")
                out['feat']=feat
                return out
            else:
                # print("Test with feature before BN")
                out['feat']=global_feat
                return out

    def load_param(self, trained_path):
        param_dict = torch.load(trained_path)
        for i in param_dict:
            if 'classifier' in i:
                continue
            self.state_dict()[i].copy_(param_dict[i])

#<font color=red>***metric***</font>

##<font color=blue>***图像***</font>

###<font color=green>***普通map***

In [ ]:

'''reid计算map函数'''
def MARKET_EVAL_FUNC(dismat,q_pids,g_pids,q_camids,g_camids,max_rank=50):
    '''
    :param dismat: 每个query与其对应查询结果的距离
    :param q_pids: 每个query行人的id
    :param g_pids: 每个查询结果的行人的id
    :param q_camids: 摄像头id，对于同一个query，如果查询结果与query都来自一个摄像头，则丢弃
    :param g_camids: 同上
    :param max_rank:
    :return:
    '''
    num_q,num_g=dismat.shape
    if num_g<max_rank:
        max_rank=num_g

    ##根据相似度进行排序
    indices=np.argsort(dismat,axis=1)

    ##在排序结果找到同id图片，用于计算cmc
    matchs=(g_pids[indices]==q_pids[:,np.newaxis]).astype(np.int32)

    all_cmc=[]
    all_AP=[]
    num_valid_q=0

    for q_idx in range(num_q):
        q_pid=q_pids[q_idx]
        q_camid=q_camids[q_idx]

        '''当前query对应查询结果的排序'''
        order=indices[q_idx]

        '''若查询结果与query同摄像机且同id，删除'''
        remove=(g_pids[order]==q_pid)&(g_camids[order]==q_camid)
        keep=np.invert(remove)

        '''计算CMC'''

        '''首先先筛选'''
        orig_cmc=matchs[q_idx][keep]

        '''全是false'''
        if not np.any(orig_cmc):
            continue

        '''累加和，因为orig_cmc里面是true，false，true为1，得到cmc列表'''
        cmc=orig_cmc.cumsum()
        cmc[cmc>1]=1
        all_cmc.append(cmc[:max_rank])
        num_valid_q+=1

        '''计算map'''

        '''预测正确的数量'''
        num_rel=orig_cmc.sum()

        '''累加和，用于计算'''
        tmp_cmc=orig_cmc.cumsum()

        '''计算准确率，前k个查询结果的准确率'''
        tmp_cmc=[x/(i+1) for i,x in enumerate(tmp_cmc)]

        '''计算召回率，召回率每次变化都是当查询结果id等于query时，故乘orig_cmc'''
        '''最终的结果类似 0,0,1,0,0,2,0,0,3这种，每个有值的都是查询正确的'''
        '''然后后面除以查询结果中同id数量，也就是上面num_rel就是找召回率'''
        '''但是这里其实已经提前乘上准确率了，最终结果类似于 0,0,1/3,0,0,2/6,0,0,3/9'''
        tmp_cmc=np.asarray(tmp_cmc)*orig_cmc

        '''以上为例，除以同id数量后，假设为五个'''
        '''0，0，1/3 /5，0，0，2/6 /5，0，0，3/9 /5'''
        '''等价于 0,0, 1/3(准确率)*1/5(召回率) ....'''
        AP = tmp_cmc.sum() / num_rel
        all_AP.append(AP)

    all_cmc = np.asarray(all_cmc).astype(np.float32)
    all_cmc = all_cmc.sum(0) / num_valid_q
    mAP = np.mean(all_AP)

    return all_cmc, mAP


class MARKET_MAP():
    def __init__(self,num_query,max_rank=50,feat_norm=True):
        '''

        :param num_query: 查询数量
        :param max_rank: 每个query限定的最大查询结果数量
        :param feat_norm:是否对特征进行l2norm
        '''
        self.num_query=num_query
        self.max_rank=max_rank
        self.feat_norm=feat_norm
        self.reset()

    def reset(self):
        '''
        feats:模型返回的特征
        pids: 图片中行人的id
        camids:图片摄像头的id
        :return:
        '''
        self.feats=[]
        self.pids=[]
        self.camids=[]

    def update(self,output):
        feat,pid,camid=output
    
        self.feats.append(feat)
        self.pids.append(pid)
        self.camids.append(camid)

    def compute(self):
        feats=torch.stack(self.feats,dim=0)
       
        if self.feat_norm:
            feats=torch.nn.functional.normalize(feats,p=2)

        '''用于查询的图片的特征及其id和摄像头id'''
        qf=feats[:self.num_query]
        q_pids=np.asarray(self.pids[:self.num_query])
        q_camids = np.asarray(self.camids[:self.num_query])

        '''每个query查询结果的特征及其id和摄像头id'''
        gf = feats[self.num_query:]
        g_pids = np.asarray(self.pids[self.num_query:])
        g_camids = np.asarray(self.camids[self.num_query:])

        m,n=qf.shape[0],gf.shape[0]
     
        '''计算qf每个query和对应查询结果的距离'''
        distmat = torch.pow(qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
                  torch.pow(gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
       
        distmat.addmm_(1, -2, qf, gf.t())
        distmat = distmat.cpu().numpy()
      
        cmc,mAP=MARKET_EVAL_FUNC(distmat,q_pids,g_pids,q_camids,g_camids)
        return cmc,mAP

###<font color=green>***rerank map***

In [ ]:
def RE_RANKING(porbFea,galFea,k1,k2,lambda_value,local_distmat=None,only_local=False):
    '''
    :param porbFea:query特征
    :param galFea: 查询结果特征
    :param k1: 一开始k近邻的k
    :param k2:
    :param lambda_value:
    :param locl_distmat:
    :param only_loca:
    :return:
    '''
    query_num=porbFea.shape[0]
    all_num=query_num+galFea.shape[0]
    if only_local:
        original_dist=local_distmat
    else:
        '''把query加进去，然后算每个图片之间的距离，用于后面reranking'''
        feat=torch.cat([porbFea,galFea])
        distmat = torch.pow(feat, 2).sum(dim=1, keepdim=True).expand(all_num, all_num) + \
                  torch.pow(feat, 2).sum(dim=1, keepdim=True).expand(all_num, all_num).t()
        distmat.addmm_(1, -2, feat, feat.t())
        original_dist = distmat.cpu().numpy()
        del feat
        if not local_distmat is None:
            original_dist = original_dist + local_distmat
    gallery_num=original_dist.shape[0]

    '''进行归一化，然后进行转置，得到归一化的距离矩阵'''
    original_dist = np.transpose(original_dist / np.max(original_dist, axis=0))

    V=np.zeros_like(original_dist).astype(np.float16)

    '''初始rank列表就是直接根据距离最小值'''
    initial_rank=np.argsort(original_dist).astype(np.int32)

    '''进行重排序'''
    '''最核心的思想就是对于query，前k个距离最近的查询结果，每一个查询结果在所有图片中的k近邻
    应该包含query，根据这个进行重排，使得rerank后尽量满足这个条件'''
    for i in range(all_num):
        '''选择当前图片的k近邻'''
        forward_k_neigh_index=initial_rank[i,:k1+1]

        '''选择k近邻的k近邻'''
        backward_k_neigh_index=initial_rank[forward_k_neigh_index,:k1+1]

        '''满足上面的条件，k近邻的k近邻包含当前图片'''
        fi=np.where(backward_k_neigh_index==i)[0]

        '''那么满足上面条件的k近邻作为rerank的候选集'''
        k_reciprocal_index=forward_k_neigh_index[fi]
        k_reciprocal_expansion_index = k_reciprocal_index

        '''如果满足上面的条件，通常就是正样本无误了，这也就是rerank的目的'''
        '''但是问题在于如果因为一些干扰，导致正样本在k1近邻之外，那rerank就没用'''
        '''所以提出一个方法进行候选集的扩充，希望能够保证正样本进入候选集'''

        '''基本的思想就是对于当前图片，我们找到了符合条件的原候选集'''
        '''那么对于候选集每一个元素，找到以一半k1为长度的满足上面条件的候选集'''
        '''如果这个候选集与原候选集的交集超过这个候选集的2/3长度，那么把这个候选集加入原候选集'''
        for j in range(len(k_reciprocal_index)):
            '''原候选集每个元素'''
            candidate=k_reciprocal_index[j]
            '''找到当前元素满足条件的候选集'''
            candidate_forward_k_neigh_index = initial_rank[candidate, :int(np.around(k1 / 2)) + 1]
            candidate_backward_k_neigh_index = initial_rank[candidate_forward_k_neigh_index,
                                               :int(np.around(k1 / 2)) + 1]
            fi_candidate = np.where(candidate_backward_k_neigh_index == candidate)[0]
            candidate_k_reciprocal_index = candidate_forward_k_neigh_index[fi_candidate]

            '''如果交集长度超过这个候选集的2/3长度'''
            if len(np.intersect1d(candidate_k_reciprocal_index,k_reciprocal_index))>(2/3)*len(candidate_k_reciprocal_index):
                k_reciprocal_expansion_index=np.append(k_reciprocal_expansion_index,candidate_k_reciprocal_index)
        k_reciprocal_expansion_index = np.unique(k_reciprocal_expansion_index)

        '''获取当前图片及其候选集的距离，进行exp'''
        weight = np.exp(-original_dist[i, k_reciprocal_expansion_index])

        '''找到每个图片与其候选集的一个权值表示，用于后面计算jaccard距离'''
        V[i, k_reciprocal_expansion_index] = weight / np.sum(weight)
    original_dist=original_dist[:query_num,]

    '''这部分处理时针对线上reid系统，因为gallery通常是一直更新的，由于计算V的过程比较麻烦'''
    '''我们只能隔一段时间计算，那么对于新加入的图片，其在V中没有值，所以采用这个近似方法来计算'''
    if k2!=1:
        V_qe=np.zeros_like(V,dtype=np.float16)
        for i in range(all_num):
            '''采用查询样本的特征均值代替上面的reciprocal编码，只是近似处理'''
            V_qe[i,:]=np.mean(V[initial_rank[i,:k2],:],axis=0)
        V=V_qe
        del V_qe
    del initial_rank
    invIndex=[]
    for i in range(gallery_num):
        '''找到候选集包含当前图片的所有query'''
        invIndex.append(np.where(V[:,i]!=0)[0])

    '''计算jaccard距离'''
    jaccard_dist=np.zeros_like(original_dist,dtype=np.float16)

    '''jaccard距离的计算，对于query q和其中一个查询样本gi的距离，我们用1-其候选集的交并比来代表，如果交并比越大，代表两者'''
    '''候选集很接近，说明两者很接近，则其距离就很小'''

    '''为了方便表示交集和并集，我们建立Vq，其中V(q,gi)代表gi是否在q候选集中，如果在为1，否则为0，那么这个Vq就是针对所有图片，类似候选集的one-hot'''
    '''Vgi同理，但是one-hot是hard编码，为了soft编码，我们使用上面np.exp(-距离）的计算'''

    '''然后交集怎么表示呢？ sum(min(vq,vgi))，就是每个位置取Vq和Vgi的最小值,然后求和，为什么呢？我们以one-hot去想，交集意味着两者一样，那么对于相同位置，只有Vq和Vgi在这个位置
    都为1，那么这个位置才为1，否则为0，所以求和的时候就是求1的个数，相当于求对应位置是相同的个数，这就是交集的定义'''

    '''并集是sum(max(Vq,Vgi)),意义同上。'''

    for i in range(query_num):
        temp_min=np.zeros(shape=[1,gallery_num],dtype=np.float16)
        '''找到当前图片的候选集'''
        indNonZero=np.where(V[i,:]!=0)[0]

        '''候选集的候选集'''
        indImages=[invIndex[ind] for ind in indNonZero]
        '''交集的计算'''
        '''不断求和，看公式'''
        for j in range(len(indNonZero)):
               temp_min[0, indImages[j]] = temp_min[0, indImages[j]] + np.minimum(V[i, indNonZero[j]],
                                                                                       V[indImages[j], indNonZero[j]])
        '''2-temp_min是并集的快速计算'''
        jaccard_dist[i] = 1 - temp_min / (2 - temp_min)


    '''作为最终的距离矩阵'''
    final_dist = jaccard_dist * (1 - lambda_value) + original_dist * lambda_value
    del original_dist
    del V
    del jaccard_dist
    final_dist = final_dist[:query_num, query_num:]
    return final_dist



'''使用reranking技术后的准确度'''
class MARKET_mAP_reranking():
    def __init__(self, num_query, max_rank=50, feat_norm=True):
        self.num_query = num_query
        self.max_rank = max_rank
        self.feat_norm = feat_norm
        self.reset()

    def reset(self):
        self.feats = []
        self.pids = []
        self.camids = []

    def update(self, output):
        feat, pid, camid = output
        self.feats.append(feat)
        self.pids.append(pid)
        self.camids.append(camid)

    def compute(self):
        feats = torch.stack(self.feats, dim=0)
        if self.feat_norm :
           
            feats = torch.nn.functional.normalize(feats, p=2)

        # query
        qf = feats[:self.num_query]
        q_pids = np.asarray(self.pids[:self.num_query])
        q_camids = np.asarray(self.camids[:self.num_query])
        # gallery
        gf = feats[self.num_query:]
        g_pids = np.asarray(self.pids[self.num_query:])
        g_camids = np.asarray(self.camids[self.num_query:])
        # m, n = qf.shape[0], gf.shape[0]
        # distmat = torch.pow(qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
        #           torch.pow(gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
        # distmat.addmm_(1, -2, qf, gf.t())
        # distmat = distmat.cpu().numpy()
        print("Enter reranking")
        distmat = RE_RANKING(qf, gf, k1=20, k2=6, lambda_value=0.3)
        cmc, mAP = MARKET_EVAL_FUNC(distmat, q_pids, g_pids, q_camids, g_camids)

        return cmc, mAP

# <font color=red>***Loss***


##<font color=blue>***分类损失***</font>

In [ ]:
class CrossEntropyLabelSmoothLoss(nn.Module):
	"""Cross entropy loss with label smoothing regularizer.
	Reference:
	Szegedy et al. Rethinking the Inception Architecture for Computer Vision. CVPR 2016.
	Equation: y = (1 - epsilon) * y + epsilon / K.
	Args:
		num_classes (int): number of classes.
		epsilon (float): weight.
	"""
	def __init__(self, num_classes, epsilon=0.1, use_gpu=True):
		super(CrossEntropyLabelSmoothLoss, self).__init__()
		self.num_classes = num_classes
		self.epsilon = epsilon
		self.use_gpu = use_gpu
		self.logsoftmax = nn.LogSoftmax(dim=1)

	def forward(self, inputs, targets):
		"""
		Args:
			inputs: prediction matrix (before softmax) with shape (batch_size, num_classes)
			targets: ground truth labels with shape (num_classes)
		"""
 
		log_probs = self.logsoftmax(inputs['logit'])
		targets = torch.zeros(log_probs.size()).scatter_(1, targets.unsqueeze(1).cpu(), 1)
		if self.use_gpu: targets = targets.cuda()
		targets = (1 - self.epsilon) * targets + self.epsilon / self.num_classes
		loss = (- targets * log_probs).mean(0).sum()
		return loss


## <font color=blue>***度量学习***

###<font color=green>***triplet***

In [ ]:
   
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

import numpy as np


def normalize(x, axis=-1):
	"""Normalizing to unit length along the specified dimension.
	Args:
	  x: pytorch Variable
	Returns:
	  x: pytorch Variable, same shape as input
	"""
	x = 1. * x / (torch.norm(x, 2, axis, keepdim=True).expand_as(x) + 1e-12)
	return x

def euclidean_dist(x, y):
	"""
	Args:
	  x: pytorch Variable, with shape [m, d]
	  y: pytorch Variable, with shape [n, d]
	Returns:
	  dist: pytorch Variable, with shape [m, n]
	"""
	m, n = x.size(0), y.size(0)
	xx = torch.pow(x.float(), 2).sum(1, keepdim=True).expand(m, n)
	yy = torch.pow(y.float(), 2).sum(1, keepdim=True).expand(n, m).t()
	dist = xx + yy
	dist.addmm_(1, -2, x.float(), y.t().float())
	dist = dist.clamp(min=1e-12).sqrt()  # for numerical stability
	return dist

def cosine_dist(x, y):
	"""
	Args:
	  x: pytorch Variable, with shape [m, d]
	  y: pytorch Variable, with shape [n, d]
	"""
	x_normed = F.normalize(x, p=2, dim=1)
	y_normed = F.normalize(y, p=2, dim=1)
	return 1 - torch.mm(x_normed, y_normed.t())

def cosine_similarity(x, y):
	"""
	Args:
	  x: pytorch Variable, with shape [m, d]
	  y: pytorch Variable, with shape [n, d]
	"""
	x_normed = F.normalize(x, p=2, dim=1)
	y_normed = F.normalize(y, p=2, dim=1)
	return torch.mm(x_normed, y_normed.t())


def hard_example_mining(dist_mat, labels, return_inds=False):
	"""For each anchor, find the hardest positive and negative sample.
	Args:
	  dist_mat: pytorch Variable, pair wise distance between samples, shape [N, N]
	  labels: pytorch LongTensor, with shape [N]
	  return_inds: whether to return the indices. Save time if `False`(?)
	Returns:
	  dist_ap: pytorch Variable, distance(anchor, positive); shape [N]
	  dist_an: pytorch Variable, distance(anchor, negative); shape [N]
	  p_inds: pytorch LongTensor, with shape [N];
		indices of selected hard positive samples; 0 <= p_inds[i] <= N - 1
	  n_inds: pytorch LongTensor, with shape [N];
		indices of selected hard negative samples; 0 <= n_inds[i] <= N - 1
	NOTE: Only consider the case in which all labels have same num of samples,
	  thus we can cope with all anchors in parallel.
	"""
	assert len(dist_mat.size()) == 2
	assert dist_mat.size(0) == dist_mat.size(1)
	N = dist_mat.size(0)

	is_pos = labels.expand(N, N).eq(labels.expand(N, N).t())
	is_neg = labels.expand(N, N).ne(labels.expand(N, N).t())

	dist_ap, relative_p_inds = torch.max(
		dist_mat[is_pos].contiguous().view(N, -1), 1, keepdim=True)
	dist_an, relative_n_inds = torch.min(
		dist_mat[is_neg].contiguous().view(N, -1), 1, keepdim=True)
	
	dist_ap = dist_ap.squeeze(1)
	dist_an = dist_an.squeeze(1)

	if return_inds:
		ind = (labels.new().resize_as_(labels)
			   .copy_(torch.arange(0, N).long())
			   .unsqueeze(0).expand(N, N))
		p_inds = torch.gather(
			ind[is_pos].contiguous().view(N, -1), 1, relative_p_inds.data)
		n_inds = torch.gather(
			ind[is_neg].contiguous().view(N, -1), 1, relative_n_inds.data)
		p_inds = p_inds.squeeze(1)
		n_inds = n_inds.squeeze(1)
		return dist_ap, dist_an, p_inds, n_inds

	return dist_ap, dist_an



class TripletHardLoss(object):
	"""Modified from Tong Xiao's open-reid (https://github.com/Cysu/open-reid).
	Related Triplet Loss theory can be found in paper 'In Defense of the Triplet
	Loss for Person Re-Identification'."""
	def __init__(self, margin=None, metric="euclidean"):
		self.margin = margin
		self.metric = metric
		if margin is not None:
			self.ranking_loss = nn.MarginRankingLoss(margin=margin)
		else:
			self.ranking_loss = nn.SoftMarginLoss()

	def __call__(self, outs, labels, normalize_feature=True):
   
		if normalize_feature:
			global_feat = normalize(outs['feat'], axis=-1)
     
		if self.metric == "euclidean":
			dist_mat = euclidean_dist(outs['feat'], global_feat)
		elif self.metric == "cosine":
			dist_mat = cosine_dist(outs['feat'], global_feat)
		else:
			raise NameError

		dist_ap, dist_an = hard_example_mining(
			dist_mat, labels)
		y = dist_an.new().resize_as_(dist_an).fill_(1)
		
		if self.margin is not None:
			loss = self.ranking_loss(dist_an, dist_ap, y)
		else:
			loss = self.ranking_loss(dist_an - dist_ap, y)
		prec = (dist_an.data > dist_ap.data).sum() * 1. / y.size(0)
		return loss

### <font color=green>***center loss***

In [ ]:

class CenterLoss(nn.Module):
    """Center loss.
    Reference:
    Wen et al. A Discriminative Feature Learning Approach for Deep Face Recognition. ECCV 2016.
    Args:
        num_classes (int): number of classes.
        feat_dim (int): feature dimension.
    """

    def __init__(self, num_classes=751, feat_dim=2048, use_gpu=True):
        super(CenterLoss, self).__init__()
        self.num_classes = num_classes
        self.feat_dim = feat_dim
        self.use_gpu = use_gpu

        if self.use_gpu:
            self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim).cuda())
        else:
            self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim))

    def forward(self, x, labels):
        """
        Args:
            x: feature matrix with shape (batch_size, feat_dim).
            labels: ground truth labels with shape (num_classes).
        """
        x=x['feat']
        assert x.size(0) == labels.size(0), "features.size(0) is not equal to labels.size(0)"

        batch_size = x.size(0)
        distmat = torch.pow(x, 2).sum(dim=1, keepdim=True).expand(batch_size, self.num_classes) + \
                  torch.pow(self.centers, 2).sum(dim=1, keepdim=True).expand(self.num_classes, batch_size).t()
        distmat.addmm_(1, -2, x, self.centers.t())

        classes = torch.arange(self.num_classes).long()
        if self.use_gpu: classes = classes.cuda()
        labels = labels.unsqueeze(1).expand(batch_size, self.num_classes)
        mask = labels.eq(classes.expand(batch_size, self.num_classes))

        dist = distmat * mask.float()
        loss = dist.clamp(min=1e-12, max=1e+12).sum() / batch_size
        #dist = []
        #for i in range(batch_size):
        #    value = distmat[i][mask[i]]
        #    value = value.clamp(min=1e-12, max=1e+12)  # for numerical stability
        #    dist.append(value)
        #dist = torch.cat(dist)
        #loss = dist.mean()
        return loss

##训练loss

# <font color=red>***lr_scheduler***</font>

## <font color=blue>***warm up***

In [ ]:
from bisect import bisect_right
import torch

class WarmupMultiStepLR:
    def __init__(
        self,
        optimizer,
        milestones,
        base_lr,
        gamma=0.1,
        warmup_factor=1.0 / 3,
        warmup_iters=500,
        warmup_method="linear",
        last_epoch=-1,
    ):
        if not list(milestones) == sorted(milestones):
            raise ValueError(
                "Milestones should be a list of" " increasing integers. Got {}",
                milestones,
            )

        if warmup_method not in ("constant", "linear"):
            raise ValueError(
                "Only 'constant' or 'linear' warmup_method accepted"
                "got {}".format(warmup_method)
            )
        self.base_lr=base_lr
        self.milestones = milestones
        self.gamma = gamma
        self.warmup_factor = warmup_factor
        self.warmup_iters = warmup_iters
        self.warmup_method = warmup_method
       


    def get_lr(self):
        warmup_factor = 1
        if self.last_epoch < self.warmup_iters:
            if self.warmup_method == "constant":
                warmup_factor = self.warmup_factor
            elif self.warmup_method == "linear":
                alpha = self.last_epoch / self.warmup_iters
                warmup_factor = self.warmup_factor * (1 - alpha) + alpha

        return self.base_lr* warmup_factor * self.gamma ** bisect_right(self.milestones, self.last_epoch)
            
        

#<font color=red>***自定义optimizer***</font>

#<font color=red>***自定义eva***

##<font color=blue>***普通***

In [ ]:
def do_eva1(cfg,model,val_loader,num_query):
  model.eval()
  device=torch.device('cuda:0')
  if cfg['rerank']:
    metric=MARKET_mAP_reranking(num_query)
  else:
    metric=MARKET_MAP(num_query)
  
  with torch.no_grad():
    for imgs,pids,cams in val_loader:
      imgs=imgs.to(device)
      outs=model(imgs)
      for out in zip(outs['feat'],pids,cams):
        
        metric.update(out)
  
  cmc,map=metric.compute()
  return cmc,map




#<font color=red>***训练辅助函数***</font>

##<font color=blue>***checkpoint***</font>

In [ ]:
class Checkpoint:
  def __init__(self,ckpt):
    self.ckpt=ckpt
    self.init=-1000
  
  def __call__(self,metric,model,optimizer):
    if metric>self.init:
      self.init=metric
      torch.save({'model':model.state_dict(),'optimizer':optimizer.state_dict()},self.ckpt)
    


#<font color=red>***自定义训练过程***</font>

##<font color=blue>strong_baseline

In [ ]:
from tqdm import tqdm
import logging
def do_train1(cfg,model,train_loader,val_loader,optimizer,scheduler,criterion,num_query,logger,checkpoint):
  device=torch.device('cuda:0')
  model=model.to(device)
  epochs=cfg['epochs']
  if cfg['amp']:
    scaler=GradScaler()
  with tqdm(total=epochs) as pbar:
    for epoch in range(epochs):
      model.train()
      scheduler.last_epoch=epoch+1
      lr=scheduler.get_lr()
      for param_group in optimizer.param_groups:
        param_group['lr'] = lr

      for images,pids in train_loader:
        loss=0
        optimizer.zero_grad()
        images=images.to(device)
        
        if cfg['amp']:
          with autocast():
            outs = model(images)
            pids=pids.to(device)
            for cri,weight in criterion:
              loss=loss+weight*cri(outs,pids)
          scaler.scale(loss).backward()
          scaler.step(optimizer)
          scaler.update()  
        else:
          outs = model(images)
          pids=pids.to(device)
          for cri,weight in criterion:
            loss=loss+weight*cri(outs,pids)
          loss.backward()
          optimizer.step()
      
      if epoch%5==0:
        cmc,map=do_eva1(cfg,model,val_loader,num_query)
        logger.info('epoch:{}---cmc:{}---map:{}----'.format(epoch,cmc,map))
        checkpoint(map,model,optimizer)
      pbar.update(1)


# <font color=red>***训练***</font>

In [ ]:
dataset=Market1501()
train_loader,val_loader,num_query,num_class=make_data_loader(dataset,cfg)


model=Baseline(num_classes)


criterion_cls=CrossEntropyLabelSmoothLoss(num_classes)
criterion_tri=TripletHardLoss(cfg['margin'])
criterion=[(criterion_cls,1.0),(criterion_tri,1.0)]


optimizer=torch.optim.Adam(model.parameters(),lr=cfg['lr'],weight_decay=cfg['weight_decay'])

scheduler = WarmupMultiStepLR(optimizer,cfg['steps'],cfg['lr'], cfg['gamma'],
               cfg['warmup_factors'],cfg['warmup_iters'], 
               'linear', 0)

ckpt=cfg['ckpt']
logpt=cfg['logpt']
logger=logging.getLogger()
fh = logging.FileHandler("spam.log")
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)
checkpoint=Checkpoint(ckpt)
do_train1(cfg,model,train_loader,val_loader,optimizer,scheduler,criterion,num_query,
          logger,checkpoint)


=> Market1501 loaded
Dataset statistics:
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


 17%|█▋        | 12/70 [15:41<1:16:40, 79.31s/it]